In [10]:
# ======================================================================
# There are 5 questions in this exam with increasing difficulty from 1-5.
# Please note that the weight of the grade for the question is relative
# to its difficulty. So your Category 1 question will score significantly
# less than your Category 5 question.
#
# Don't use lambda layers in your model.
# You do not need them to solve the question.
# Lambda layers are not supported by the grading infrastructure.
#
# You must use the Submit and Test button to submit your model
# at least once in this category before you finally submit your exam,
# otherwise you will score zero for this category.
# ======================================================================
# QUESTION
#
# Build and train a neural network to predict sunspot activity using
# the Sunspots.csv dataset.
#
# Your neural network must have an MAE of 0.12 or less on the normalized dataset
# for top marks.
#
# Code for normalizing the data is provided and should not be changed.
#
# At the bottom of this file, we provide  some testing
# code in case you want to check your model.

# Note: Do not use lambda layers in your model, they are not supported
# on the grading infrastructure.
# To get the maximum score, your model must have an MAE OF .12 or less.

import csv
import tensorflow as tf
import numpy as np
import urllib

In [11]:
# DO NOT CHANGE THIS CODE
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    ds = ds.shuffle(shuffle_buffer)
    ds = ds.map(lambda w: (w[:-1], w[1:]))
    return ds.batch(batch_size).prefetch(1)

In [12]:
data_url = 'https://github.com/dicodingacademy/assets/raw/main/Simulation/machine_learning/sunspots.csv'
urllib.request.urlretrieve(data_url, 'sunspots.csv')

time_step = []
sunspots = []

with open('sunspots.csv') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    for row in reader:
        sunspots.append(float(row[2]))
        time_step.append(int(row[0]))

series = np.array(sunspots)

In [13]:
sunspots

[96.7,
 104.3,
 116.7,
 92.8,
 141.7,
 139.2,
 158.0,
 110.5,
 126.5,
 125.8,
 264.3,
 142.0,
 122.2,
 126.5,
 148.7,
 147.2,
 150.0,
 166.7,
 142.3,
 171.7,
 152.0,
 109.5,
 105.5,
 125.7,
 116.7,
 72.5,
 75.5,
 94.0,
 101.2,
 84.5,
 110.5,
 99.7,
 39.2,
 38.7,
 47.5,
 73.3,
 58.3,
 83.3,
 118.3,
 98.8,
 99.5,
 66.0,
 130.7,
 48.8,
 45.2,
 77.7,
 62.7,
 66.7,
 73.3,
 53.3,
 76.2,
 63.3,
 60.0,
 52.8,
 36.7,
 65.0,
 46.7,
 41.7,
 33.3,
 11.2,
 0.0,
 5.0,
 2.8,
 22.8,
 34.5,
 44.5,
 31.3,
 20.5,
 13.7,
 40.2,
 22.0,
 7.0,
 17.0,
 18.7,
 11.3,
 10.8,
 0.0,
 0.0,
 14.3,
 5.3,
 29.7,
 39.5,
 11.3,
 33.3,
 20.8,
 11.8,
 9.0,
 15.7,
 20.8,
 21.5,
 6.0,
 10.7,
 19.7,
 23.8,
 28.3,
 15.7,
 23.5,
 35.3,
 43.7,
 50.0,
 63.5,
 21.3,
 41.7,
 85.5,
 66.2,
 54.2,
 107.8,
 55.8,
 62.7,
 86.7,
 81.7,
 120.5,
 77.3,
 75.0,
 73.3,
 64.5,
 104.2,
 62.8,
 71.7,
 71.7,
 80.5,
 73.3,
 78.0,
 78.3,
 81.7,
 83.3,
 85.0,
 118.8,
 128.7,
 99.5,
 77.2,
 95.0,
 112.2,
 99.2,
 124.5,
 97.2,
 120.0,
 80.5,
 110.0,


In [14]:
# Normalization Function.
#DO NOT CHANGE THIS CODE
min = np.min(series)
max = np.max(series)
series -= min
series /= max
time = np.array(time_step)

# DO NOT CHANGE THIS CODE
split_time = 3000

time_train = time[:split_time]
x_train = series[:split_time]
time_valid = time[split_time:]
x_valid = series[split_time:]

In [15]:
# DO NOT CHANGE THIS CODE
window_size = 30
batch_size = 32
N_FEATURES = 1
shuffle_buffer_size = 1000


train_set = windowed_dataset(x_train, window_size=window_size, batch_size=batch_size,
                             shuffle_buffer=shuffle_buffer_size)

val_set = windowed_dataset(x_valid, window_size=window_size, batch_size=batch_size,
                             shuffle_buffer=shuffle_buffer_size)

In [16]:
class MyCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('val_mae') < 0.12:
            print("\nSTOP!")
            self.model.stop_training = True
#MY CODE HERE.

In [17]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv1D(16, 3, activation='relu', padding='causal', input_shape=[window_size, 1]),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=True)),
    tf.keras.layers.Dense(1)]) # MY CODE

In [18]:
optimizer_modified = tf.keras.optimizers.Adam(learning_rate=0.001)

In [19]:
model.compile(loss='huber', optimizer=optimizer_modified, metrics=['mae'])

In [20]:
model.fit(train_set, validation_data=val_set, callbacks = MyCallback(), epochs=100)

Epoch 1/100
     93/Unknown - 8s 10ms/step - loss: 0.0041 - mae: 0.0627
STOP!
93/93 [==============================] - 10s 23ms/step - loss: 0.0041 - mae: 0.0627 - val_loss: 0.0013 - val_mae: 0.0341
